This notebook is for creating summary plots of stream metabolism parameters estimated from dissolved oxygen time series using the BASEmetab package of Grace et al. (2015). In-situ measurements collected from East Fork Creek in Franklin, TN by John C. Ayers, Weizhuo Jing, Kevin Chen, Ella Daugherty, and Gabriel Perez.  
Reference:
Grace MR, Giling DP, Hladyz S, et al (2015) Fast processing of diel oxygen curves: Estimating stream metabolism with BASE (BAyesian Single-station Estimation). Limnol. Oceanogr. methods / 13:e10011

In [3]:
# Let's read the CSV file using pandas
import pandas as pd
# Read the CSV file using pandas (we won't set the working directory as it's not needed in this environment)
data_file_path = "EFC_StreamMetabolismSummary.csv"
data = pd.read_csv(data_file_path, parse_dates=["Date"], dayfirst=False)

# Display the first few rows of the dataset as a summary
data.head()

,Date,Device,GPP.mean,GPP.sd,GPP.median,ER.mean,ER.sd,ER.median,NEP.mean,NEP.sd,NEP.median,PR.mean,PR.sd,PR.median,K.mean,K.sd,K.median
0,2022-05-24,MiniDOT,9.724921,0.397755,9.718426,24.543829,1.016071,24.540043,-14.818908,0.681979,-14.815466,0.396303,0.007375,0.396179,9.444339,0.405853,9.434647
1,2022-05-25,MiniDOT,8.379095,0.353796,8.366785,26.166950,1.059195,26.162258,-17.787855,0.742909,-17.782109,0.320234,0.005074,0.320231,11.687572,0.469452,11.682121
2,2022-05-26,MiniDOT,7.016506,0.506425,7.000212,32.861249,2.302127,32.816748,-25.844743,1.853357,-25.812503,0.213585,0.006465,0.213687,14.972180,1.058652,14.938265
3,2022-06-04,Exo2,14.578490,0.542414,14.574772,19.262054,0.773292,19.257909,-4.683564,0.359292,-4.670982,0.757041,0.012617,0.756872,8.885572,0.328489,8.881961
4,2022-06-05,Exo2,12.011935,0.439199,12.006980,15.501951,0.622592,15.497732,-3.490016,0.306524,-3.487460,0.775103,0.014233,0.775022,6.800502,0.257384,6.796968


In [7]:
# Reshape data from wide to long format
data_long = data.melt(id_vars=['Date', 'Device'], value_vars=['GPP.mean', 'ER.mean', 'NEP.mean'], 
                      var_name='Parameter', value_name='Value')
data_long['Parameter'] = data_long['Parameter'].str.split('.').str[0]

# For each parameter, calculate max and min values based on standard deviations and the mean values
data_long['max'] = 0
data_long['min'] = 0

for index, row in data_long.iterrows():
    original_index = index // 3  # Because we reshaped 3 columns to long format
    if row['Parameter'] == 'GPP':
        data_long.at[index, 'max'] = row['Value'] + data.loc[original_index, 'GPP.sd']
        data_long.at[index, 'min'] = row['Value'] - data.loc[original_index, 'GPP.sd']
    elif row['Parameter'] == 'ER':
        data_long.at[index, 'max'] = row['Value'] + data.loc[original_index, 'ER.sd']
        data_long.at[index, 'min'] = row['Value'] - data.loc[original_index, 'ER.sd']
    else:
        data_long.at[index, 'max'] = row['Value'] + data.loc[original_index, 'NEP.sd']
        data_long.at[index, 'min'] = row['Value'] - data.loc[original_index, 'NEP.sd']

# Filter required columns for plotting
data_plot = data_long[['Date', 'Device', 'Parameter', 'Value', 'max', 'min']]

# Display the first few rows of the transformed data
data_plot.head()


,Date,Device,Parameter,Value,max,min
0,2022-05-24,MiniDOT,GPP,9.724921,10.122676,9.327166
1,2022-05-25,MiniDOT,GPP,8.379095,8.776850,7.981339
2,2022-05-26,MiniDOT,GPP,7.016506,7.414261,6.618751
3,2022-06-04,Exo2,GPP,14.578490,14.932286,14.224694
4,2022-06-05,Exo2,GPP,12.011935,12.365731,11.658139


### Make plot of Gross Primary Productivity GPP versus Ecosystem Respiration ER

In [10]:
data_long

,Date,Device,Parameter,Value,max,min
0,2022-05-24,MiniDOT,GPP,9.724921,10.122676,9.327166
1,2022-05-25,MiniDOT,GPP,8.379095,8.776850,7.981339
2,2022-05-26,MiniDOT,GPP,7.016506,7.414261,6.618751
3,2022-06-04,Exo2,GPP,14.578490,14.932286,14.224694
4,2022-06-05,Exo2,GPP,12.011935,12.365731,11.658139
...,...,...,...,...,...,...
91,2023-01-09,MiniDOT,NEP,-5.460153,-4.871366,-6.048940
92,2023-01-10,MiniDOT,NEP,-5.377430,-4.788643,-5.966217
93,2023-01-11,MiniDOT,NEP,-7.238766,-6.729565,-7.747966
94,2023-01-12,MiniDOT,NEP,-15.324663,-14.815462,-15.833864


In [13]:
import matplotlib.pyplot as plt

# Filter out data with ER > 50
# data_filtered = data_plot[data_plot['ER.mean'] < 50]

# Calculate error bars for plotting
data_plot['xlow'] = data_plot['GPP.mean'] - data_plot['GPP.sd']
data_plot['xhigh'] = data_plot['GPP.mean'] + data_plot['GPP.sd']
data_plot['ylow'] = data_plot['ER.mean'] - data_plot['ER.sd']
data_plot['yhigh'] = data_plot['ER.mean'] + data_plot['ER.sd']

# Plotting using matplotlib
plt.figure(figsize=(10, 8))
colors = {'MiniDOT': 'blue', 'Exo2': 'orange'}
for device, color in colors.items():
    subset = data_plot[data_plot['Device'] == device]
    plt.errorbar(subset['GPP.mean'], subset['ER.mean'], 
                 xerr=[subset['GPP.mean'] - subset['xlow'], subset['xhigh'] - subset['GPP.mean']],
                 yerr=[subset['ER.mean'] - subset['ylow'], subset['yhigh'] - subset['ER.mean']],
                 fmt='o', color=color, label=device)

plt.plot([0, 40], [0, 40], color='gray', linestyle='--')  # Add line with slope 1
plt.xlim(0, 40)
plt.ylim(0, 40)
plt.xlabel("GPP mean (mg O2 L-1 d-1)")
plt.ylabel("ER mean (mg O2 L-1 d-1)")
plt.legend()
plt.title("GPP vs ER with Error Bars")
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()

# Save the plot (we'll save it as PNG for display purposes, but SVG can be used for high-resolution plots)
plot_filename = "/mnt/data/EFC_GPPvsER_ErrorBars.png"
plt.savefig(plot_filename, format='png')

# Display the plot
plt.show()

plot_filename

KeyError: 'GPP.mean'